<a href="https://colab.research.google.com/github/tyukios/genai/blob/main/final_RPG/trpg_new2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -fsSL https://ollama.ai/install.sh | sh
!nohup ollama serve &
!ollama pull gemma3:4b
!ollama pull llama3.2
!ollama pull mistral
!ollama pull deepseek-r1:1.5b
!pip install openai gradio

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.5 MB/s eta 0:00:00


In [2]:
import openai
from openai import OpenAI
api_key = "ollama"
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
最終穩定修正版Ollama故事生成器
"""

import gradio as gr
import os
import json
import time
from datetime import datetime
import random
from openai import OpenAI
import asyncio
from typing import List, Dict, Any, Tuple

class StoryLLM:
    def __init__(self):
        self.client = OpenAI(
            api_key="ollama",
            base_url="http://localhost:11434/v1"
        )
        self.available_models = ["gemma3:4b","llama3.2", "mistral", "deepseek-r1:1.5b"]
        self.current_model = "gemma3:4b"
        self.story_cache = {}
        self.response_times = []

        self.genre_prompts = {
            "奇幻": "你是一位擅長奇幻故事的作家，對神祕世界的架構、魔法體系與種族文化如數家珍。你擅於編織充滿神話與史詩感的世界，描繪王權爭奪、禁忌魔法與靈魂契約之間的張力。你的筆下世界往往富含古老預言與宿命對抗，每個角色都有命運牽引的深刻羈絆與情感掙扎，並透過五感營造魔法的觸感、召喚的氣味與禁術的陰影。",
            "科幻": "你是一位科幻小說專家，擅長構築未來文明與科技進化的張力，對AI意識、宇宙殖民、時空交錯與人機邊界有深入理解。你擁有哲學與物理學的跨領域視角，文字裡融合冷靜邏輯與人性的渴望，常探討技術背後的倫理矛盾與情感演化。你的故事帶有蒸汽龐克、賽博龐克或後人類主義色彩，擅於描繪科技觸感與末日氛圍下的微妙情感流動。",
            "懸疑": "你是一位懸疑大師，擅長佈局精巧、節奏緊湊的推理結構。你筆下的故事充滿心理操控、真假錯亂與潛藏的秘密，讀者永遠無法信任每一個角色的話語或行為。你熟稔犯罪心理與人性的黑暗面，擅用意象鋪陳緊繃氛圍，透過微小線索建構層層反轉。你的角色內心世界往往有著難以名狀的創傷與慾望，讓懸疑與情感糾葛共存。",
            "愛情": "你是一位浪漫小說作家，擅長描繪深刻而細膩的情感波動，筆下人物真摯動人、關係充滿張力與火花。你擅於建構戀人間的心理差距、慾望交鋒與曖昧拉扯，無論是青澀初戀、禁忌戀情，或歷經傷痛的成熟之愛，都能用詩意語言觸動讀者心弦。你喜歡透過觸感、氣味、聲音來描寫每一次靠近與疏離，情慾場景具張力又充滿象徵性。",
            "冒險": "你是一位冒險故事專家，專精於描寫穿越未知、踏上征途的驚險歷程。你筆下的世界總有隱藏的寶藏、危險的地形與背叛的同伴，而角色在每一場生死關頭中蛻變。你擅長描繪極端環境下的肢體感受、心理承受力與人性的試煉，讓冒險不只是地理遷移，更是心靈的自我追尋與成長。你的敘事中充滿動態感與臨場感，讀者彷彿也親身踏上旅途。",
            "歷史": "你是一位歷史小說作家，擅於重現古代文明的氛圍與日常細節，從朝代更替、宮廷陰謀到市井小民的命運，都描寫得入木三分。你對服飾、器物、語言與禮制有考據癖，善於將史實與虛構揉合，透過角色的視角讓歷史「活」起來。你筆下的世界充滿權力、階級、愛情與悲劇，每個選擇都可能改變時代洪流的方向。",
            "恐怖": "你是一位恐怖大師，專長於描繪心理陰影與潛意識的怪物。你筆下的恐懼不僅來自外在鬼魅，更來自角色內心深處無法逃避的創傷與慾望。你擅用氣味、聲響、視覺扭曲與身體異變營造不安，故事常在真實與幻覺邊界遊走，引導讀者直視人性深層的瘋狂與孤寂。你的語言具有詩意與噩夢般的象徵性，恐怖成為揭示靈魂傷痕的一種藝術。"
        }

    async def generate_with_ollama_async(self, prompt: str, system_prompt: str = None) -> str:
        """安全地呼叫Ollama API"""
        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        messages.append({"role": "user", "content": prompt})

        start_time = time.time()
        try:
            response = await asyncio.to_thread(
                self.client.chat.completions.create,
                model=self.current_model,
                messages=messages,
                temperature=0.7,
                max_tokens=800
            )
            elapsed = time.time() - start_time
            self.response_times.append(elapsed)
            print(f"生成完成 (耗時: {elapsed:.2f}s)")
            return response.choices[0].message.content
        except Exception as e:
            print(f"API請求失敗: {str(e)}")
            return f"⚠️ 故事生成失敗，請檢查Ollama服務\n錯誤: {str(e)}"

    async def generate_story_start(self, genre: str, custom_prompt: str = "") -> Dict[str, Any]:
        """安全地生成故事開頭"""
        try:
            cache_key = f"{genre}_{custom_prompt}"
            if cache_key in self.story_cache:
                return self.story_cache[cache_key]

            system_prompt = self.genre_prompts.get(genre, "")
            if custom_prompt:
                system_prompt += f"\n特別要求: {custom_prompt}"

            user_prompt = f"""請用繁體中文創作一個{genre}故事的開頭：
1. 【標題】(使用Markdown二級標題)
2. 主角簡介 (1-2句話)
3. 初始場景 (1-2句話)
4. 提供3個行動選項 (格式: 選項1: [描述])

請保持內容簡潔，總長度不超過15句話。"""

            full_response = await self.generate_with_ollama_async(user_prompt, system_prompt)

            # 安全處理回應
            story_text = "\n".join([line for line in full_response.split("\n") if not line.startswith("選項")])
            options = [line.split(": ")[1] for line in full_response.split("\n") if line.startswith("選項") and ": " in line]

            if not options or len(options) < 3:
                options = ["探索周圍環境", "與附近角色交談", "檢查隨身物品"]

            result = {
                "genre": genre,
                "story_text": story_text,
                "options": options[:3]
            }
            self.story_cache[cache_key] = result
            return result
        except Exception as e:
            print(f"故事生成異常: {str(e)}")
            return {
                "genre": genre,
                "story_text": f"⚠️ 故事生成時發生錯誤\n{str(e)}",
                "options": ["重試", "換一個類型", "檢查服務"]
            }

    async def continue_story(self, history: List[Dict], user_input: str) -> Dict[str, Any]:
        """安全地繼續故事"""
        try:
            if not history:
                return await self.generate_story_start("奇幻")

            last_part = history[-1]

            prompt = f"""繼續以下{last_part['genre']}故事:

當前情節:
{last_part['story_text']}

讀者選擇: {user_input}

請:
1. 發展合理後續 (1-2段話)
2. 保持原有風格
3. 提供3個新選項 (格式: 選項1: [行動])
4. 總長度不超過20句話"""

            full_response = await self.generate_with_ollama_async(
                prompt,
                system_prompt=self.genre_prompts.get(last_part["genre"], "")
            )

            # 安全處理回應
            story_text = "\n".join([line for line in full_response.split("\n") if not line.startswith("選項")])
            options = [line.split(": ")[1] for line in full_response.split("\n") if line.startswith("選項") and ": " in line]

            if not options or len(options) < 3:
                options = ["進一步探索", "與新角色互動", "嘗試解決當前問題"]

            return {
                "genre": last_part["genre"],
                "story_text": story_text,
                "options": options[:3]
            }
        except Exception as e:
            print(f"故事繼續異常: {str(e)}")
            return {
                "genre": history[-1]["genre"] if history else "奇幻",
                "story_text": f"⚠️ 無法繼續故事\n{str(e)}",
                "options": ["重試", "回到上一步", "重新開始"]
            }

def create_gradio_interface():
    story_gen = StoryLLM()

    with gr.Blocks(title="互動故事生成器", theme=gr.themes.Soft()) as demo:
        current_story = gr.State([])
        is_generating = gr.State(False)

        with gr.Column():
            gr.Markdown("## 🏰 互動故事生成器")

            with gr.Row():
                with gr.Column(min_width=200):
                    model_selector = gr.Dropdown(
                        story_gen.available_models,
                        value="gemma3:4b",
                        label="🛠️ 選擇模型",
                        interactive=True
                    )
                    genre = gr.Dropdown(
                        list(story_gen.genre_prompts.keys()),
                        value="奇幻",
                        label="📖 故事類型",
                        interactive=True
                    )
                    custom_prompt = gr.Textbox(
                        label="✏️ 客製化要求 (可選)",
                        placeholder="例如：主角是會說話的貓",
                        lines=2
                    )
                    start_btn = gr.Button("🚀 開始故事", variant="primary")

                with gr.Column():
                    story_display = gr.Markdown(
                        label="📜 故事內容",
                        value="**請選擇故事類型後點擊「開始故事」**",
                        elem_id="story-box"
                    )

            choice_row = gr.Row(visible=False)
            with choice_row:
                options_radio = gr.Radio(
                    label="🛤️ 你的選擇",
                    choices=[],
                    interactive=True,
                    elem_id="options-radio"
                )
                continue_btn = gr.Button("⏭️ 繼續故事", variant="primary")

            status = gr.Textbox(
                label="🔄 狀態",
                value="準備就緒",
                interactive=False
            )

        async def start_story(genre, custom_prompt, model):
            story_gen.current_model = model

            # 返回所有輸出組件的初始狀態
            initial_outputs = {
                status: "🔄 正在生成故事...",
                story_display: "**正在創作你的故事，請稍候...**",
                options_radio: gr.Radio(choices=[], interactive=False),
                choice_row: gr.Row(visible=False),
                current_story: [],
                is_generating: True
            }
            yield initial_outputs

            try:
                story = await story_gen.generate_story_start(genre, custom_prompt)

                # 返回所有輸出組件的最終狀態
                final_outputs = {
                    status: f"✅ 故事已生成 (耗時: {story_gen.response_times[-1]:.1f}s)" if story_gen.response_times else "✅ 故事已生成",
                    story_display: story["story_text"],
                    options_radio: gr.Radio(
                        choices=story["options"],
                        value=None,
                        interactive=True
                    ),
                    choice_row: gr.Row(visible=True),
                    current_story: [story],
                    is_generating: False
                }
                yield final_outputs
            except Exception as e:
                error_outputs = {
                    status: "❌ 生成失敗",
                    story_display: f"⚠️ 故事生成失敗\n錯誤: {str(e)}",
                    options_radio: gr.Radio(choices=["重試", "換一個類型"], value=None),
                    choice_row: gr.Row(visible=True),
                    current_story: [],
                    is_generating: False
                }
                yield error_outputs

        async def continue_story(history, choice, is_gen):
            if is_gen:
                return

            if not choice:
                yield {
                    status: "⚠️ 請先選擇一個行動",
                    story_display: history[-1]["story_text"] if history else "請先開始故事",
                    options_radio: gr.Radio(
                        choices=history[-1]["options"] if history else [],
                        value=None,
                        interactive=True
                    ),
                    choice_row: gr.Row(visible=True),
                    current_story: history,
                    is_generating: False
                }
                return

            # 顯示載入狀態
            yield {
                status: "🔄 正在發展故事...",
                story_display: history[-1]["story_text"] + "\n\n**正在創作後續情節...**",
                options_radio: gr.Radio(choices=[], interactive=False),
                choice_row: gr.Row(visible=False),
                current_story: history,
                is_generating: True
            }

            try:
                new_part = await story_gen.continue_story(history, choice)
                updated_history = history + [new_part]

                yield {
                    status: f"✅ 情節已更新 (耗時: {story_gen.response_times[-1]:.1f}s)" if story_gen.response_times else "✅ 情節已更新",
                    story_display: new_part["story_text"],
                    options_radio: gr.Radio(
                        choices=new_part["options"],
                        value=None,
                        interactive=True
                    ),
                    choice_row: gr.Row(visible=True),
                    current_story: updated_history,
                    is_generating: False
                }
            except Exception as e:
                yield {
                    status: "❌ 繼續故事失敗",
                    story_display: f"⚠️ 無法繼續故事\n錯誤: {str(e)}",
                    options_radio: gr.Radio(
                        choices=["重試", "回到上一步"],
                        value=None
                    ),
                    choice_row: gr.Row(visible=True),
                    current_story: history,
                    is_generating: False
                }

        start_btn.click(
            start_story,
            inputs=[genre, custom_prompt, model_selector],
            outputs=[status, story_display, options_radio, choice_row, current_story, is_generating]
        )

        continue_btn.click(
            continue_story,
            inputs=[current_story, options_radio, is_generating],
            outputs=[status, story_display, options_radio, choice_row, current_story, is_generating]
        )

    return demo

if __name__ == "__main__":
    # 檢查Ollama服務
    try:
        print("正在檢查Ollama服務...")
        test_client = OpenAI(
            api_key="ollama",
            base_url="http://localhost:11434/v1",
            timeout=10
        )
        models = test_client.models.list()
        print(f"可用模型: {[m.id for m in models.data]}")
    except Exception as e:
        print(f"❌ Ollama連接失敗: {str(e)}")
        print("請確保:")
        print("1. 已安裝Ollama: curl -fsSL https://ollama.ai/install.sh | sh")
        print("2. 已啟動服務: ollama serve")
        print("3. 已下載模型: ollama pull gemma3:4b")
        exit(1)

    # 啟動界面
    print("正在啟動Gradio界面...")
    demo = create_gradio_interface()

    demo.launch(
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True
    )


正在檢查Ollama服務...
可用模型: ['deepseek-r1:1.5b', 'mistral:latest', 'llama3.2:latest', 'gemma3:4b']
正在啟動Gradio界面...
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://50c6b0af78c1cde224.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
